<font size="5px"> <a href="https://jaydeepsb.github.io/deepdive/">Go to Home page</a></font>

# Motif finding problem Motif (k-mer) given NxL strings of Dna strands

In [196]:
import numpy as np
from itertools import permutations, product
from collections import Counter
from skimage.util.shape import view_as_windows
from copy import deepcopy

import pylab as plt

Steps:

* Read the list of sequences
* For each sequence, reshape sequence into possible number of k-mers in that strings times k (nucleotides)
* To start, select randomly 1 k-mer per sequence, by omitting 1 sequence at random. to form a motif matrix.
* Generate nucleotide count for each place-holder in the motif (i.e. 4xk count matrix)
* Add pseudo counts (+1) to the whole matrix if there is one or more zero counts.
* Generate profile-most motif, i.e., probability distribution based motif
* Match it with k-mers in the string that was omitted. Calculate entropy and pick the highest valued k-mer for the motif matrix.
* Repeat.

In [326]:
def seq2arr(seq):
    return np.array(list(seq)).astype(object)

def hamming_distance(chaine1, chaine2):
    return sum(c1 != c2 for c1, c2 in zip(chaine1, chaine2))

def motifFinder(listofseq, k, d, iterations=3):
    N = len(listofseq)
    L = len(listofseq[0])
    seq_list = list(map(lambda s:s.upper(), listofseq ))
    long_seq = np.empty((0,k), object)
    for i in np.arange(N):
        #long_seq[i] = view_as_windows(seq2arr(long_seq[i]), k,1)
        long_seq = np.vstack((long_seq, view_as_windows(seq2arr(seq_list[i]), k,1)))
    print(long_seq.shape)
    long_seq = long_seq.reshape((N,L-k+1,k)) #(no. of sequences x no. of k-mers x k)
    
    #==== Generate motif matrix ===============
    #n_start_idxs = np.random.choice(np.arange(L-k+1), size=N-1)
    n_start_idxs = np.random.choice(np.arange(L-k+1), size=N)

    motif_matrix_full =deepcopy(long_seq[np.arange(N), n_start_idxs, :])
    
    skip_idx = np.random.choice( np.arange(N), size=1)[0]  
    m=0
    while m <= iterations:
        #==== Skip 1 sequence at random
        if m == 0:
            skip_idx = np.random.choice( np.arange(N), size=1)[0]
        else:
            skip_idx = np.random.choice( np.setdiff1d(np.arange(N), skip_idx), size=1)[0]
            pass
        include_idxs =  np.setdiff1d(np.arange(N), skip_idx)

        #motif_matrix = long_seq[include_idxs, n_start_idxs, :]
        motif_matrix = motif_matrix_full[include_idxs, :]
        print("Motif matrix after iteration: ", m)
        print(motif_matrix_full.sum(axis=1).reshape((-1,1)))

        bases = {"A":0, "T":1,"G":2,"C":3}
        profile_matrix = np.zeros((4, k))
        #==== Count nucleotides
        for i in np.arange(k):
            c = Counter({"A":0, "T":0,"G":0,"C":0})
            c.update(motif_matrix[:,i])
            profile_matrix[:,i] = [c.get(key) for key in bases.keys()]
        #==== Add pseudo counts ===================
        if np.any(profile_matrix == 0):
            profile_matrix += 1
        #==== Create Motif profile ===================
        profile_matrix /= profile_matrix.sum(axis=0)

        #==== Create profile-most Motif ===================
        profile_most_motif = [np.random.choice(list(bases.keys()), size=1, p=profile_matrix[:,i])[0] for i in np.arange(k)]
        #Entropy of the profile_most_motif
        pm_entropy = 0.0
        for j in np.arange(k):
            h = bases.get(profile_most_motif[j])
            pm_entropy -= profile_matrix[h,j]*np.log(profile_matrix[h,j])
        #============================================================
        
        #calculate probabilities of all k-mers in the skipped sequence
        prob_mat = np.zeros(((L-k+1),k))
        skipped_seq = long_seq[skip_idx,:,:]
        for i in np.arange(L-k+1):
            kmer = long_seq[skip_idx,i,:]
            kmer = [bases.get(key) for key in kmer]
            for j in np.arange(k):
                prob_mat[i,j] = profile_matrix[kmer[j],j]
        prob_mat = np.prod(prob_mat, axis=1).flatten()
        prob_mat /= prob_mat.sum()
        #print(prob_mat)
        #==== Randomly choose one kmer from above kmers, given their probabilities
        probable_kmer_idx = np.random.choice(np.arange(L-k+1), size=1, p=prob_mat)
        #id_mx = np.argmax(prob_mat)
        most_matched_kmer = long_seq[skip_idx,probable_kmer_idx,:]
        motif_matrix_full[skip_idx,:] = most_matched_kmer
        #print(skip_idx,id_mx, ''.join(most_matched_kmer))
        print("Prob.-most motif: ", ''.join(profile_most_motif), " entropy: ", np.round(pm_entropy,4))
        print("===============================================")
        m += 1
    #return ''.join(profile_most_motif)

In [327]:
sequence_list= [
    "ttACCTtaac",
    "gATGTctgtc",
    "ccgGCGTtag",
    "cactaACGAg",
    "cgtcagAGGT",
]

k=4
d=3

motifFinder(sequence_list, k, d, iterations=2)


(35, 4)
Motif matrix after iteration:  0
[['ACCT']
 ['TGTC']
 ['CGTT']
 ['ACGA']
 ['TCAG']]
Prob.-most motif:  CCCT  entropy:  1.2997
Motif matrix after iteration:  1
[['ACCT']
 ['TGTC']
 ['GGCG']
 ['ACGA']
 ['TCAG']]
Prob.-most motif:  AGCA  entropy:  1.4288
Motif matrix after iteration:  2
[['ACCT']
 ['TGTC']
 ['GGCG']
 ['ACGA']
 ['TCAG']]
Prob.-most motif:  GTAG  entropy:  1.213
